<a href="https://colab.research.google.com/github/Achillesy/DeepTICI/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepTICI
---
This repository  is a fork of <https://github.com/IPMI-ICNS-UKE/DeepTICI>.
Thanks to STROKE [publication](https://doi.org/10.1161/STROKEAHA.120.033807) for sharing the official code and model weights.

The following code follows the license rights and limitations of the original shared code (CC BY-NC 4.0). You can view the full license text here:
<https://raw.githubusercontent.com/IPMI-ICNS-UKE/DeepTICI/main/LICENSE.txt>


**This code is provided for research and educational purposes only. Please do not use it for commercial purposes.**

If you have any questions, please contact me at: xuchu_liu@rush.edu

## Requirements

In [1]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=6902f3dea8b3061f5da1927bc80524c767204a9c413b3b663aa75a29283c2bf4
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


## Installation

In [3]:
!wget https://github.com/IPMI-ICNS-UKE/DeepTICI/raw/main/model_weights/experiment_I
!wget https://github.com/IPMI-ICNS-UKE/DeepTICI/raw/main/model_weights/experiment_III_0
!wget https://github.com/IPMI-ICNS-UKE/DeepTICI/raw/main/model_weights/experiment_III_1
!wget https://github.com/IPMI-ICNS-UKE/DeepTICI/raw/main/model_weights/experiment_III_2

--2023-12-13 16:17:25--  https://github.com/IPMI-ICNS-UKE/DeepTICI/raw/main/model_weights/experiment_I
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IPMI-ICNS-UKE/DeepTICI/main/model_weights/experiment_I [following]
--2023-12-13 16:17:25--  https://raw.githubusercontent.com/IPMI-ICNS-UKE/DeepTICI/main/model_weights/experiment_I
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59091797 (56M) [application/octet-stream]
Saving to: ‘experiment_I.1’

experiment_I.1      100%[===================>]  56.35M   286MB/s    in 0.2s    

2023-12-13 16:17:26 (286 MB/s) - ‘experiment_I.1’ saved [59091797/5909

## Load model and parameters

## Please upload your files

In [ ]:
import os
import pydicom

import matplotlib.pyplot as plt

## Clean up temporary files